In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from utils.preprocess import *
from utils.model import *

### Reduced ts

In [3]:
produv_train = pd.read_csv(config['data_path'] + 'produv_train.csv')
gas_train = pd.read_csv(config['data_path'] + 'gas_train.csv')
chronom_train = pd.read_csv(config['data_path'] + 'chronom_train.csv')

produv_test = pd.read_csv(config['data_path'] + 'produv_test.csv')
gas_test = pd.read_csv(config['data_path'] + 'gas_test.csv')
chronom_test = pd.read_csv(config['data_path'] + 'chronom_test.csv')

In [4]:
reduce_ts(ts_preproc(gas_train, 'Time'), chronom_train).to_csv(config['data_path'] + 'reduced_gas_train.csv')
reduce_ts(ts_preproc(produv_train, 'SEC'), chronom_train).to_csv(config['data_path'] + 'reduced_produv_train.csv')
reduce_ts(ts_preproc(gas_test, 'Time'), chronom_test).to_csv(config['data_path'] + 'reduced_gas_test.csv')
reduce_ts(ts_preproc(produv_test, 'SEC'), chronom_test).to_csv(config['data_path'] + 'reduced_produv_test.csv')

### Extract ts features

In [11]:
train_data = read_data('train')

In [12]:
X_train = preprocess_data(train_data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2063/2063 [01:36<00:00, 21.33it/s]


In [13]:
test_data = read_data('test')
for file in files:
    test_data[file] = test_data[file][test_data[file].NPLV != 512905]

In [14]:
X_test = preprocess_data(test_data)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 779/779 [00:31<00:00, 24.39it/s]


In [56]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

X_train = X_train.select_dtypes(include=numerics)
X_test  = X_test.select_dtypes(include=numerics)

In [10]:
#X_train.to_csv(config['data_path'] + 'num_extracted_features_train.csv')
# X_test.to_csv(config['data_path'] + 'num_extracted_features_test.csv')
# X_train = pd.read_csv(config['data_path'] + 'num_extracted_features_train.csv')
# X_test  = pd.read_csv(config['data_path'] + 'num_extracted_features_test.csv')

In [60]:
train_NPLV = X_train['NPLV']
test_NPLV = X_test['NPLV']
selected_features_train, selected_features_test = ts_select_features(X_train.drop(['NPLV'], axis = 1), X_test.drop(['NPLV'], axis = 1))
# selected_features_train, selected_features_test = X_train.drop(['NPLV'], axis = 1), X_test.drop(['NPLV'], axis = 1)

In [61]:
# selected_features_test = selected_features_test.drop(['Unnamed: 0'], axis = 1)
selected_features_test['NPLV'] = test_NPLV

In [62]:
imposter = pd.DataFrame(selected_features_test.mean(),columns=[779]).T
imposter['NPLV'] = [512905]

In [63]:
selected_features_test = pd.concat([selected_features_test, imposter]).sort_values('NPLV').reset_index().drop(['index'],axis=1)

In [69]:
with open('clustering_test.pickle', 'rb') as f:
    clustering_test = pickle.load(f)
    
    clustering_test_df = pd.DataFrame({'NPLV' : selected_features_test['NPLV'].values, 'cluster_label' : clustering_test})
    selected_features_test = selected_features_test.merge(clustering_test_df, on='NPLV')

In [70]:
# selected_features_train = selected_features_train.drop(['Unnamed: 0'], axis = 1)
selected_features_train['NPLV'] = train_NPLV

with open('clustering_train.pickle', 'rb') as f:
    clustering_train = pickle.load(f)
    
    clustering_train_df = pd.DataFrame({'NPLV' : train_NPLV.values, 'cluster_label' : clustering_train})
    selected_features_train = selected_features_train.merge(clustering_train_df, on='NPLV')

In [71]:
chronom_data_train = pd.read_csv('data_chronom/chronom_duration_train.csv')
chronom_data_test = pd.read_csv('data_chronom/chronom_duration_test.csv')

KeyError: 'NPLV'

In [72]:
chronom_data_train

,NOP_Вхождение в гр.МНЛЗ,NOP_Завалка лома,NOP_Заливка чугуна,NOP_Наведение гарнисажа,NOP_Нагрев лома,NOP_Наложение продувки,NOP_Неиспр. электр. обор,NOP_Обрыв горловины,NOP_Осмотр конвертера,NOP_Отсутствие O2,NOP_Отсутствие чугуна,NOP_Подварка футеровки,NOP_Полусухое торкрет.,NOP_Продувка,NOP_Слив шлака,NOP_межпл. простой
0,0,16,203,100,730,0,0,0,144,126,0,0,0,1170,46,666
1,620,31,188,112,788,0,0,0,198,440,0,0,0,1230,118,845
2,0,133,209,136,738,0,0,0,102,0,0,0,0,1226,55,614
3,0,17,175,184,776,0,0,541,0,0,0,0,0,1058,42,1401
4,0,136,184,133,736,0,0,0,120,0,0,0,0,1239,49,929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,0,102,242,94,0,0,0,0,0,0,0,0,0,1063,296,940
2059,0,1415,224,160,0,0,0,0,63,0,2648,0,0,1108,33,1530
2060,0,534,315,182,0,0,0,438,62,0,0,0,0,1058,199,885
2061,462,645,160,167,0,0,0,0,38,0,104,0,0,1169,150,908


In [ ]:
selected_features_train = selected_features_train.merge(chronom_data_train, on='NPLV')
selected_features_test = selected_features_test.merge(chronom_data_test, on='NPLV')

In [66]:
selected_features_train.shape

(2063, 5402)

In [67]:
selected_features_train.to_csv(config['data_path'] + 'preprocessed_train.csv')
selected_features_test.to_csv(config['data_path'] + 'preprocessed_test.csv')

### Preprocess plavki

In [4]:
target = read_target_train()
train_data = read_data('train')
test_data = read_data('test')

In [5]:
from category_encoders import TargetEncoder

def first_preprocess_plavki(plavki):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    
    plavki = plavki.copy()
    
    new_plavki = pd.DataFrame()
    for NPLV in plavki.NPLV.unique():
        cur_ts = plavki[plavki['NPLV'] == NPLV]
        new_ts = cur_ts.iloc[0:1,:]
        new_ts[new_ts.select_dtypes(include=numerics).columns] = pd.DataFrame(cur_ts[new_ts.select_dtypes(include=numerics).columns].mean(), columns = new_ts.index).T

        new_plavki = pd.concat((new_plavki, new_ts))
    new_plavki = new_plavki.reset_index().drop(['index'], axis = 1)
    
    new_plavki['plavka time'] = (pd.to_datetime(new_plavki['plavka_VR_KON']) - pd.to_datetime(new_plavki['plavka_VR_NACH'])).dt.seconds
    new_plavki = new_plavki.drop(['plavka_VR_NACH', 'plavka_VR_KON'], axis = 1)
    return new_plavki


In [6]:
def preprocess_plavki(train_plavki, test_plavki):
    new_plavki_train = first_preprocess_plavki(train_plavki)
    new_plavki_test = first_preprocess_plavki(test_plavki)
    
    plavki_cat = ['plavka_NMZ', 'plavka_NAPR_ZAD', 'plavka_TIPE_FUR', 'plavka_TIPE_GOL']
    for col in plavki_cat:
        new_plavki_train[col + '_C'] = new_plavki_train[col]
        new_plavki_test[col + '_C']  = new_plavki_test[col] 
        new_plavki_train[col + '_TST'] = new_plavki_train[col]
        new_plavki_test[col + '_TST']  = new_plavki_test[col]
        
    target = read_target_train()
    
    encoder_C = TargetEncoder(cols = list(map(lambda x: x + '_C', plavki_cat)))
    encoder_TST = TargetEncoder(cols = list(map(lambda x: x + '_TST', plavki_cat)))
    encoder_C.fit(new_plavki_train, target['C'])
    encoder_TST.fit(new_plavki_train, target['TST'])
    
    new_plavki_train = encoder_C.transform(new_plavki_train)
    new_plavki_test = encoder_C.transform(new_plavki_test)
    new_plavki_train = encoder_TST.transform(new_plavki_train)
    new_plavki_test = encoder_TST.transform(new_plavki_test)
    
    return new_plavki_train, new_plavki_test
    

In [7]:
new_plavki_train, new_plavki_test = preprocess_plavki(train_data['plavki'], test_data['plavki'])

In [8]:
new_plavki_train.to_csv(config['data_path'] + 'preprocessed_plavki_train.csv')
new_plavki_test.to_csv(config['data_path'] + 'preprocessed_plavki_test.csv')